In [ ]:
import os
import datetime
import numpy as np
import xarray as xr
import glob
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import WetSpa_main_model_fuctions as WetSpa
import WetSpa_static_preproc as WetSpa_prepros

In [ ]:
#Lines taken from PixSWAB
# multiprocessing client
#applications are broken into smaller routines that run independently,resulting in imporved performance 
#memory_limit=48GB 
client = WetSpa.start_multiprocessing()
client.restart()

### Main folder of input and output files
##### Specify the start and end date for the model run (which usually corresponds to the input files start and end date)

In [ ]:
#lines below are taken from PixSWAB
MAIN_FOLDER = r''
# Create output folder with the time and date
time_now = datetime.datetime.now()
time_str = time_now.strftime('%Y_%m_%d_%Hh_%Mm')
output_dir = 'output_dir_'+str(time_str)
dir_out = os.path.join(MAIN_FOLDER,'output', 'nc', output_dir)
dir_plots = os.path.join(MAIN_FOLDER,'output', 'plots', output_dir)

if not os.path.exists(dir_out):
    os.makedirs(dir_out)
    
if not os.path.exists(dir_plots):
    os.makedirs(dir_plots)

log_file_path = os.path.join(dir_out, 'log_file.txt')
log_file = WetSpa._log_create(log_file_path)

Startdate='2010-06-01'
Enddate='2017-05-31'#'2018-05-31'

log_file.write('{:>26s}: {}\n'.format('Start date', str(Startdate)))
log_file.write('{:>26s}: {}\n'.format('End date', str(Enddate)))

In [ ]:
#Calibration parameters 

#correction factor for PET 
#range: close to 1 (0.3-2.0)
k_ep=0.62

#exponent refelction the effect of rainfall intensity or actual surface runoff coefficient when rainfal intesnity is very small
#range: 0.01 -15.00
k_run=4.51

# threshold of rainfall intensity
#range: 1-700
P_max= 220.77

#soil moisture ratio relative to the field capacity for setting up the initial soil moisture content
#range: negative or 0.1 - 3.0 
k_ss = 0.30

#inital GW storage in depth [mm]
#range: 1-500
g0 = 262.30

#maximum GW storage in depth [mm]
g_max = 575.59

#interflow scaling factor reflecting the effect of organic material and root system in the topsoil layer on the horizontal hydraulic conductivity
#range: 0.1-20
ki = 3.11

#time interval in h 
dt_hours = 720

#GW flow recession coefficient (reflects GW recession regime for entire catchment)
#range: 1*10^-6 to 1*10^-1
kg = 0.028

#following parameters are currently not used in the model, but can be implemented 
#base temoerature for estimating snowmelt, in which the precipitation shifts from rain to snow (negative value)
T0 = -1.000
#rainfall degree-day coefficient determining the rate of snowmelt caused by rainfall (negative value)
k_rain = 0.00
#temperature degree-day coefficient calculating snowmelt (negative value)
k_snow = 2.00

#Parameters for the incoorporation of waterbodies and irrigated land use (crop coefficient and irrigation efficiency)
#kc for water bodies -> this is to incooperate the ET of water bodies 
kcw= 0.8 #0.95#1.05

#kc for irrigated forest 
kcf= 0.2

#irrigation efficiency -> this depends on the irrigation method that is predomantely used in the study area 
ieff= 0.7

csv_lcc_info = os.path.join(MAIN_FOLDER,"input/csvs/ETref.csv")
df=pd.read_csv(csv_lcc_info,sep=',',index_col=0)
ET0_dict = df.T.to_dict('list')
log_file.write('{:>26s}: {}\n'.format('LCC info file path', str(csv_lcc_info)))

#Lines below are taken from PixSWAB
period = {
         's': Startdate,
         'e': Enddate
         }
wdir=r''

input_files =  { # SB! sub-catchment
                'p_in' : os.path.join(MAIN_FOLDER,"input/maps/p_CHIRPS_monthly.nc"),
                 'lu_in' : os.path.join(MAIN_FOLDER,"input/maps/lcc_WetSpa.nc"),
                'et_act_in' : os.path.join(MAIN_FOLDER,"input/maps/et_SSEBop_monthly.nc"),
                 'pet_in' : os.path.join(MAIN_FOLDER,"input/maps/ret_monthly.nc"),
                'tmin_in' : os.path.join(MAIN_FOLDER,"input/maps/Tmin_monthly.nc"),
                'tmax_in' : os.path.join(MAIN_FOLDER,"input/maps/Tmax_monthly.nc"),
                 'DEM' : os.path.join(MAIN_FOLDER,"input\maps\K3_DEM.tif"), 
                'soil' : os.path.join(MAIN_FOLDER,"input\maps\K3_soil_final.tif")
                }
parameters = {
    'k_ep':k_ep,
    'k_run':k_run,
    'P_max':P_max,
    'k_ss':k_ss,
    'ki':ki,
    'dt_hours':dt_hours,
    'kg':kg,
    'g0':g0,
    'g_max':g_max,
    'T0':T0,
    'k_rain':k_rain,
    'k_snow':k_snow,
    'kcw': kcw,
    'kcf': kcf,
    'ieff': ieff
    }

#all possible outputs: 
output = ['ET', 'SRO', 'ET_diff', 'ET_diff_noirr', 'ETg', 'ETb', 'Sup', 'ETi', 'ETs', 'ETd', 'ETirr', 'SM', 'ET_per', 'ET_per_noirr']

#output = ['ET_nirr']

# encoding for writing dataset to file
latchunk = 200
lonchunk = 200
timechunk = 1
chunks = {'time':timechunk,'latitude':latchunk, 'longitude':lonchunk}

# read the sizes of the precipitation dataset
with xr.open_dataset(input_files['p_in']) as xds:
    latchunk = min(latchunk,xds.latitude.size)
    lonchunk = min(lonchunk,xds.longitude.size)
    xds.close()

# chunks = [timechunk,latchunk, lonchunk]
comp = dict(zlib=True, complevel=9, least_significant_digit=2, chunksizes=list(chunks.values()))  

encoding_output = dict(
                dtype =  np.float32,
                _FillValue = np.nan,
                scale_factor = np.float32(1.0),
                add_offset = np.float32(0.0),
                zlib = True,
                complevel = 9,
                least_significant_digit = 2,
                chunksizes=tuple(chunks.values())
            )

wetspa_params = {
    'period':period,
    'wdir':wdir, 
    'input_files':input_files,
    'parameters':parameters,
    'chunks':chunks,
    'output':output,
    'ET0_dict':ET0_dict
  }

log_file.write('{:>26s}:\n'.format('Input files and parameters'))
# for key, value in input_files.items():
#     log_file.write('{:>26s}: {}\n'.format(key, str(value)))
# for key, value in parameters.items():
#     log_file.write('{:>26s}: {}\n'.format(key, str(value)))

#### Run the static preprocessing 

In [ ]:
preproc=WetSpa_prepros.static_preproc(wetspa_params)

#### Run the WetSpa model 

In [ ]:
result = WetSpa.wetspa_model(wetspa_params)

In [ ]:
result

#### Write the output  to netCDF files

In [ ]:
#lines below are taken from PixSWAB 
from dask.distributed import progress
import time
t1 = time.perf_counter () #returns value (in fractional seconds) of a performance counter 

log_file.write('{:>26s}:\n'.format('Output files path'))
print("writing the netcdf file")   
for r in result: #outputs that were previously selected 
    if(r is not None):
        #print(r)
        print("* {0}".format(r.name))
        nc_fn=r.name+'.nc'
        nc_path=os.path.join(dir_out,nc_fn)
        encoding = {r.name: encoding_output}
        
#         r.to_netcdf(nc_path,encoding=encoding)
#         r.close()

        delayed_obj = r.to_netcdf(nc_path,encoding=encoding, compute=True)
        result = delayed_obj#.persist()
        progress(result)
        r.close()
        name = r.name
        log_file.write('{:>26s}: {}\n'.format(name, str(nc_path)))
        # del r
print(f"Writing the netCDF is completed!")
#why is the time important in this step? Time was not measured/taken into consideration for the other steps 
t2 = time.perf_counter ()
time_diff = t2 - t1
print(f"\nIt took {time_diff} Secs to execute this method")
WetSpa._log_close(log_file)

In [ ]:
#lines below are orginally taken from PixSWAB and adjusted 
def nc2ts(bf_sr, shape):
    ts_all=[]
    for nc in bf_sr: #insert paths to your BaseFlow and SRO output netCDF files
        var,name = WetSpa.open_nc(nc,chunks)
        print('*',name)   #nothing is printed as this is only loading the function

        #Clip the datset by shape of the subbasins  
        var = var.rio.set_crs("epsg:4326", inplace=True)
        var_clipped = var.rio.clip(shape.geometry.values, shape.crs, drop=False)

        # Calculate area of the subbasins
        area = WetSpa.area_sqkm(var_clipped[0])

        # compute volume by multiplying the depth by area
        Volume=var_clipped*area*1e-6
        Volume = Volume.rename('{0}'.format(var.name))
        # sum the volume spatially and convert it dataframe to have timeseries values
        ts = Volume.sum(dim=['latitude','longitude'], skipna=True).to_dataframe()
        ts_all.append(ts)
        var.close()
        var_clipped.close()
    return ts_all
    
#print(ts_all) #-> calculation of bf does not seem to be correct -> always 0 (does not change when d (gw store time cons) changes)
         
infiles = [input_files['p_in']]#,input_files['e_in']
outfiles = glob.glob(os.path.join(dir_out,'*.nc'))

#files_to_read = ['Base_flow','Surface_Runoff','Incremental_ET_M','Rainfall_ET_M']
files_to_read = ['Surface_Runoff']#'Base_flow',
x = pd.Series(outfiles)
bf_sr = x[x.str.contains('|'.join(files_to_read))]
bf_sr = infiles+bf_sr.to_list()

In [ ]:
#lines below are orginally taken from PixSWAB and adjusted
# Lolsur
print('Calculating monthly fluxes volume for Lolsur')  

# change the shapefile here

shapef = os.path.join(MAIN_FOLDER,"input\shapefile\Lolsur_basin.shp")
shape = gpd.read_file(shapef,crs="EPSG:4326")
shape = shape.to_crs("EPSG:4326")

ts_all = nc2ts(bf_sr, shape)
df = pd.concat(ts_all, axis =1)  

# read observed outflow
path_obs = os.path.join(MAIN_FOLDER,"input/csvs/discharge_Lolsur_monthly.csv")
Q_m=pd.read_csv(path_obs,sep=',',index_col=0,skiprows=0)

Q_m.index=[datetime.datetime.strptime(y,'%d/%m/%Y') for y in Q_m.index]#definition of time was different bevor but gave errors 
#Q_m = (Q_d*3600*24/1e9).resample('MS').sum()
#Q_m['Lolsur_Q_in_cumecs'] = Q_m['Lolsur_Q_in_cumecs']*3600*24/1e9*Q_m.index.days_in_month
#Q_m.rename(columns={"Lolsur_Q_in_cumecs":"Lolsur_Q_in_km3/month"}, inplace=True)
Q_m = Q_m[df.index[0]:df.index[-1]]

#read additional inflow for K3 
path_obs = os.path.join(MAIN_FOLDER,"input/csvs/inflow_K3_monthly.csv")
Q_m_K3=pd.read_csv(path_obs,sep=',',index_col=0,skiprows=0)
Q_m_K3.index=[datetime.datetime.strptime(y,'%d/%m/%Y') for y in Q_m_K3.index]
#Q_m_K3['K3_Q_in_m3/s'] = Q_m_K3['K3_Q_in_m3/s']*3600*24/1e9*Q_m_K3.index.days_in_month
#Q_m_K3.rename(columns={"K3_Q_in_m3/s":"K3_Q_in_km3/month"}, inplace=True)
Q_m_K3 = Q_m_K3[df.index[0]:df.index[-1]]

df2= pd.concat([df, Q_m, Q_m_K3], axis =1)
#df2 = df2.dropna()

#add additional time series for the plotting 
NSE = WetSpa.nash_sutcliffe(df2['Lolsur_Q_in_km3/month'], df2['Surface_Runoff_M'] + df2['K3_Q_in_km3/month'])
df3 = df2.dropna()
KGE = WetSpa.kge(df3['Lolsur_Q_in_km3/month'], df3['Surface_Runoff_M']+df3['K3_Q_in_km3/month'])

plt.figure(figsize=(12,8))
#plt.rcParams["font.family"] = "Times New Roman"
ax=plt.subplot(211)
ax.plot(df2['Lolsur_Q_in_km3/month'], label='Observed discharge at Lolsur')
ax.plot(df2.index, df2['Surface_Runoff_M'] + df2['K3_Q_in_km3/month'], label='Modeled discharge')
#ax.plot(df2.index, df2['K3_Q_in_km3/month'], label='Inflow K3')
ax.set_ylabel('$km^3/month$')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

ax.set_xlim([df2.index[0], df2.index[-1]])

leg1 =  plt.legend()
plt.title('KGE = {0:.4f}, NSE = {1:.4f}'.format(KGE,NSE))

ax2 = ax.twinx()
width = 5
ax2.bar(df2.index, df2['precipitation'],  width=width, color='g', label = "Rainfall [km3/month]",  alpha = 0.9)
ax2.set_ylabel('Rainfall', color='g',  fontsize=8)
ax2.tick_params('y', colors='g')
ax2.invert_yaxis()
leg2 = ax2.legend(loc='center right', fontsize=8)
ax2.tick_params(labelsize=8)

plt.legend(leg2.get_patches()+leg1.get_lines(), 
        [text.get_text() for text in leg2.get_texts()+leg1.get_texts()], 
        loc='upper left', fancybox=False, framealpha=0.3, shadow=False, borderpad=0)
leg1.remove()

ax=plt.subplot(212)
ax.plot(np.cumsum(df2['Lolsur_Q_in_km3/month']), label='Cumulative Observed discharge')
ax.plot(df2.index, np.cumsum(df2['Surface_Runoff_M']+df2['K3_Q_in_km3/month']), label='Cumulative modeled discharge')
#ax.plot(df2.index, np.cumsum(df2['K3_Q_in_km3/month']), label='Cumulative inflow K3')

ax.set_ylabel('Flow [$km^3/month$]')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlim([df2.index[0], df2.index[-1]])
plt.legend()
plt.savefig(os.path.join(dir_plots, 'Modeled_vs_observed_discharge_Lolsur.png'), bbox_inches='tight', dpi=600)

In [ ]:
#from matplotlib.ticker import FixedLocator, FixedFormatte
shapef = os.path.join(MAIN_FOLDER,"input\shapefile\Lolsur_basin.shp")
shape = gpd.read_file(shapef,crs="EPSG:4326")
shape = shape.to_crs("EPSG:4326")

ts_all = nc2ts(bf_sr, shape)
df = pd.concat(ts_all, axis =1)  

# read observed outflow
path_obs = os.path.join(MAIN_FOLDER,"input/csvs/discharge_Lolsur_monthly.csv")
Q_m=pd.read_csv(path_obs,sep=',',index_col=0,skiprows=0)

Q_m.index=[datetime.datetime.strptime(y,'%d/%m/%Y') for y in Q_m.index]#definition of time was different bevor but gave errors 
Q_m = Q_m[df.index[0]:df.index[-1]]

#read additional inflow for K3 
path_obs = os.path.join(MAIN_FOLDER,"input/csvs/inflow_K3_monthly.csv")
Q_m_K3=pd.read_csv(path_obs,sep=',',index_col=0,skiprows=0)
Q_m_K3.index=[datetime.datetime.strptime(y,'%d/%m/%Y') for y in Q_m_K3.index]
Q_m_K3 = Q_m_K3[df.index[0]:df.index[-1]]

df2= pd.concat([df, Q_m, Q_m_K3], axis =1)

plt.figure(figsize=(12,8))
#plt.rcParams["font.family"] = "Times New Roman"
ax=plt.subplot(211)
ax.plot(df2['Lolsur_Q_in_km3/month'], label='Observed discharge at Lolsur')
ax.plot(df2.index, df2['Surface_Runoff_M'] + df2['K3_Q_in_km3/month'], label='Modeled discharge')
#ax.plot(df2.index, df2['K3_Q_in_km3/month'], label='Inflow K3')
ax.set_ylabel('$km^3/month$', fontsize = 14)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlim([df2.index[0], datetime.date(2017, 5, 31)])
plt.rcParams['font.size'] = 12
ax.set_xlabel('Years', fontsize = 14)
#ax.set_xticklabels(ax.get_xticklabels(), fontsize = 12)
#ax.set_yticklabels(ax.get_yticklabels(), fontsize = 12)

title = plt.title('Observed vs. modeled discharge in Lolsur', fontsize = 14)
leg1 =  plt.legend(fontsize = 12)
plt.savefig(os.path.join(dir_plots, 'Time_Series_WetSpa.png'), bbox_inches='tight', dpi=600)

In [ ]:
#Outflow of WetSpa is used as input for WA+ sheet generation:
df['Discharge']=df2['Surface_Runoff_M'] + df2['K3_Q_in_km3/month']
df['test_out_discharge'] = df['Discharge']*1000000
df = df.dropna()

df['Inflow']= df2['K3_Q_in_km3/month']
df['test_out_inflow'] = df['Inflow']*1000000
df = df.dropna()

df.to_csv(os.path.join(MAIN_FOLDER,'K3_inflow.csv'), sep = ';', columns = ['test_out_inflow'], header = ['Inflow'])
df.to_csv(os.path.join(MAIN_FOLDER,'outflow.csv'), sep = ';', columns = ['test_out_discharge'], header = ['Discharge'])

### Calculating an average tif file for nc files  

In [ ]:
import rasterio
files =  { # SB! sub-catchment
                'e_diff' : os.path.join(dir_out, "Evapotranspiration_percentage_no_irrigation_M.nc")
}
ETdiff,_= WetSpa.open_nc(files['e_diff'],chunks)
average=ETdiff.mean(dim='time', skipna=True)
average_values =np.flipud(average)

data = average_values
lon = average.coords['longitude'].values
lat = average.coords['latitude'].values


transform = rasterio.transform.from_bounds(lon.min(), lat.min(), lon.max(), lat.max(), len(lon), len(lat))
with rasterio.open(os.path.join(dir_out, "ET_percentage_average_no_irr.tif"), 'w', driver='GTiff', height=data.shape[0], width=data.shape[1],
                       count=1, dtype=data.dtype, crs='EPSG:4326', transform=transform) as dst:
        dst.write(data, 1)
print(average)

In [ ]:
# For the whole basin -> do not delete this part!!! 
#Output need to be changed above first -> calculation of Incremental_ET and Rainfall_ET is needed! 
print('Calculating monthly fluxes volume for Burhanpur')  

infiles = [input_files['p_in'],input_files['e_in']]
outfiles = glob.glob(os.path.join(dir_out,'*.nc'))

files_to_read = ['Base_flow','Surface_Runoff','Incremental_ET_M','Rainfall_ET_M']
#files_to_read = ['Base_flow','Surface_Runoff']
x = pd.Series(outfiles)
bf_sr = x[x.str.contains('|'.join(files_to_read))]
bf_sr = infiles+bf_sr.to_list()


# change the shapefile here

shapef = os.path.join(MAIN_FOLDER,"input\shapefile\K3.shp")
shape = gpd.read_file(shapef,crs="EPSG:4326")
shape = shape.to_crs("EPSG:4326")

ts_all = nc2ts(bf_sr, shape)
df = pd.concat(ts_all, axis =1)  


ts = df.dropna()
fig, ax = plt.subplots(figsize=(12,4));
ax.plot(ts['precipitation'], color='lightblue', label = 'precipitation');
ax.plot(ts['evapotranspiration'], color='k', label = 'evapotranspiration');
ax.set_xlim([ts.index[0], ts.index[-1]])
labels = ['blue ET', 'green ET']
ax.stackplot(ts.index, ts['Incremental_ET_M'], ts['Rainfall_ET_M'], labels = labels, colors=['royalblue','limegreen']);
plt.legend()
plt.savefig(os.path.join(dir_plots, 'et_green_blue1.png'), bbox_inches='tight', dpi=600)

fig, ax = plt.subplots(figsize=(12,4));
plt.plot(ts['Incremental_ET_M'], color='royalblue', label = 'blue ET');
plt.plot(ts['Rainfall_ET_M'], color='limegreen', label = 'green ET');
labels = ['precipitation', 'evapotranspiration']
ax.stackplot(ts.index,abs(ts['precipitation'] - ts['evapotranspiration']), color='grey')
ax.set_xlim([ts.index[0], ts.index[-1]])
plt.plot(ts['precipitation'] - ts['evapotranspiration'], 'k--', label = 'water yield')
plt.legend()
plt.savefig(os.path.join(dir_plots, 'et_green_blue2.png'), bbox_inches='tight', dpi=600)
